In [ ]:
import glob
import numpy as np
import cv2
import os

a) Mục tiêu / tác vụ mà bộ dữ liệu hướng tới là gì?

### Mục tiêu của bộ dữ liệu là xây dựng một mô hình phân loại để nhận diện bệnh nhiễm khuẩn phổi (PNEUMONIA) từ ảnh X-Quang. Mô hình sẽ dự đoán một trong hai nhãn: "NORMAL" (bình thường) và "PNEUMONIA" (bệnh).

b) Kích thước của dữ liệu là bao nhiêu trên các tập train, dev và test?

In [ ]:
IMG_SIZE = 227
classes = ['NORMAL', 'PNEUMONIA']

def load_dataset(path):
    X = []
    y = []
    
    for c in classes:
        class_path = os.path.join(path, c)
        files = glob.glob(class_path + "/*.jpeg")
        for file in files:
            img = cv2.imread(file)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img.flatten()
            X.append(img)
            y.append(c)
    X = np.array(X)
    y = np.array(y)
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    return X, y_encoded

# Đường dẫn tới các thư mục train, dev, test
train_path = "/kaggle/input/chest-xray/chest_xray/train"
val_path = "/kaggle/input/chest-xray/chest_xray/val"
test_path = "/kaggle/input/chest-xray/chest_xray/test"

# Đọc dữ liệu
X_train, y_train = load_dataset(train_path)
X_val, y_val = load_dataset(val_path)
X_test, y_test = load_dataset(test_path)

# Kích thước dữ liệu
print("Train set size:", X_train.shape)
print("Val set size:", X_val.shape)
print("Test set size:", X_test.shape)



c) Phân bố nhãn theo từng tác vụ trên các tập train, dev và test. Vẽ biểu đồ barplot thể hiện phân bố nhãn trên từng tập dữ liệu.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_label_distribution(y_train, y_val, y_test):
    train_labels, train_counts = np.unique(y_train, return_counts=True)
    val_labels, val_counts = np.unique(y_val, return_counts=True)
    test_labels, test_counts = np.unique(y_test, return_counts=True)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    sns.barplot(x=train_labels, y=train_counts, ax=axes[0])
    axes[0].set_title("Train set label distribution")
    sns.barplot(x=val_labels, y=val_counts, ax=axes[1])
    axes[1].set_title("val set label distribution")
    sns.barplot(x=test_labels, y=test_counts, ax=axes[2])
    axes[2].set_title("Test set label distribution")

    plt.show()

plot_label_distribution(y_train, y_val, y_test)


d) Hãy hiển thị khoảng 3 bức hình về ảnh X-Quang phổi lên màn hình.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_samples(X, y, num_samples=3):
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    indices = np.random.choice(range(X.shape[0]), num_samples, replace=False)
    for i, idx in enumerate(indices):
        axes[i].imshow(X[idx].reshape((IMG_SIZE, IMG_SIZE, 3)), cmap='gray')
        label = 'NORMAL' if y[idx] == 0 else 'PNEUMONIA'
        axes[i].set_title(f"Label: {label}")
        axes[i].axis('off')
    plt.show()

# Hiển thị mẫu ảnh
show_samples(X_train, y_train)


Bài 2: Huấn luyện mô hình bằng phương pháp Logistic Regression

In [ ]:
from sklearn.utils import resample
def balance_dataset(X, y):
    # Tạo mảng chứa chỉ mục của mỗi lớp
    indices_per_class = [np.where(y == i)[0] for i in np.unique(y)]
    
    # Tìm chỉ mục của lớp có số lượng mẫu ít nhất
    min_samples = min([len(indices) for indices in indices_per_class])
    
    # Downsample từng lớp về số lượng mẫu ít nhất
    X_balanced = []
    y_balanced = []
    for indices in indices_per_class:
        downsampled_indices = resample(indices, replace=False, n_samples=min_samples, random_state=42)
        X_balanced.extend(X[downsampled_indices])
        y_balanced.extend(y[downsampled_indices])
    
    # Chuyển đổi thành mảng numpy
    X_balanced = np.array(X_balanced)
    y_balanced = np.array(y_balanced)
    
    return X_balanced, y_balanced

X_train_balanced, y_train_balanced = balance_dataset(X_train, y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler

def scaler_data(X1, X2, X3):
    scaler = StandardScaler()
    X1 = scaler.fit_transform(X1)
    X2 = scaler.fit_transform(X2)
    X3 = scaler.fit_transform(X3)
    return X1, X2, X3

X_train_balanced, X_val_, X_test_ = scaler_data(X_train_balanced, X_val, X_test)



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Huấn luyện mô hình
model_lr = LogisticRegression(penalty="elasticnet", solver='saga', l1_ratio=0.25, C=0.5, tol = 0.008)
model_lr.fit(X_train_balanced, y_train_balanced)

y_test_predict_lr = model_lr.predict(X_test_)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Logistic Regression model accuracy:",
      accuracy_score(y_test, y_test_predict_lr))

print("Confusion Matrix:\n",
     confusion_matrix(y_test, y_test_predict_lr))

print("Logistic Regression model report:\n", 
      classification_report(y_test, y_test_predict_lr, target_names=classes))


In [ ]:
import joblib


In [ ]:
joblib.dump(model_lr, 'LogisticRegression_model.pkl')

Bài 3: Huấn luyện mô hình bằng phương pháp SVM

In [ ]:
model_svc = SVC()
model_svc.fit(X_train_balanced, y_train_balanced)

y_test_predict_svc = model_svc.predict(X_test_)
print("SVC model accuracy:",
      accuracy_score(y_test, y_test_predict_svc))

print("Confusion Matrix:\n",
     confusion_matrix(y_test, y_test_predict_svc))

print("SVC model report:\n", 
      classification_report(y_test, y_test_predict_svc, target_names=classes))


In [ ]:
joblib.dump(model_svc, 'SVC_model.pkl')